# Set-up

This notebook contains predictive modeling for various distinct target options, namely:

3 Modeling: Binary Classification (**PRICE_BINARY**)
- Binary split is set as a USD Price amount in the Dashboard

4 Modeling: Multi-Class Classification (**PRICE_CLASS**)
- TBD

5 Modeling: Regression (**PRICE_LOG**)
- TBD

**As a consequence, the respective targets need to be selected via the Dashboard BEFORE running the corresponding section.**

## Libraries and Dashboard

In [88]:
# Import libraries

# General
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import math
import joblib

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline  # Same, but with the latter it is not necessary to name estimator and transformer
#from imblearn.pipeline import Pipeline as Imb_Pipe
from sklearn.compose import ColumnTransformer
from sklearn.cluster import DBSCAN

# Feature Selection
from sklearn.feature_selection import SelectKBest, chi2, f_classif, GenericUnivariateSelect, mutual_info_classif
#import eli5

# Predictive Modeling (Models)
from sklearn.dummy import DummyClassifier, DummyRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_predict, cross_val_score, cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, NuSVC, SVR
from sklearn.linear_model import LinearRegression, LogisticRegression, PassiveAggressiveRegressor, ElasticNet, SGDRegressor, RANSACRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor, VotingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, IsolationForest
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from xgboost import XGBClassifier, XGBRegressor
from scipy.stats import randint
from sklearn.multiclass import OneVsRestClassifier
from catboost import CatBoostRegressor

# Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer, fbeta_score, accuracy_score, confusion_matrix, f1_score, precision_recall_curve, recall_score, precision_score, roc_auc_score
from scipy.sparse import csr_matrix
import scipy.stats as stats

# Neural Networks
from keras import models, layers, optimizers, regularizers
from keras.utils.vis_utils import model_to_dot
from keras.wrappers.scikit_learn import KerasRegressor
from IPython.display import SVG

In [89]:
# Dashboard (Global Variables)
dataset_loc = "berlin"              # "berlin", "paris", "amsterdam"
dataset_date = "2020-03-17"         # "2019-12-11", "2020-01-10", "2020-02-18", "2020-03-17", "2020-05-14"
target = 'price_log'             # for regression: 'occupancy_rate', 'price_log' | for classification: 'price_class', 'occupancy_class'
binary_split = 50                   # price at which "price_binary" will be split
# !Important: Please select features for prediction in cell "Feature selection" below!
scoring = 'neg_median_absolute_error'  # for regression: 'neg_mean_squared_error', 'r2', 'neg_mean_poisson_deviance', 'neg_median_absolute_error' | for classification: "f1(_micro, _macro, _weighted for multiclass)", "recall", "precision", "accuracy", "roc_auc"
test_size = 0.2
random_state = 42
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [90]:
# Import data_engineered
data = pd.read_pickle(f"saves/{dataset_loc}_{dataset_date}/data_engineered.pkl")

In [91]:
# Feature selection

#... by removing certain features
all_features = [
    el for el in data.columns if el not in [
        'occupancy_rate', 'occupancy_class', 'listing_no', 'price_log',
        'price_class', 'price_binary', "review_scores_class_new",
        "review_scores_class", "review_scores_calc", "neighbourhood_cleansed"
    ]
]

#... by only considering certain features
key_features = [
    "accommodates_per_bed", "am_balcony", "am_breakfast", "am_child_friendly",
    "am_elevator", "am_essentials", "am_pets_allowed", "am_private_entrance",
    "am_smoking_allowed", "am_tv", "bathrooms_log", "bedrooms",
    "calc_host_lst_count_sqrt_log", "cancellation_policy",
    "guests_included_calc", "host_is_superhost", "instant_bookable",
    "maximum_nights", "minimum_nights_sqrt", "property_type", "room_type",
    "wk_mth_discount", "zipcode"
]

# select features for predictive modeling from above: [all_features, key_features]
pred_features = key_features

#Display columns:
#all_features
#key_features

In [92]:
# Display target and used features
# Print current setting for TARGET
target_upper = target.upper()
print(f"You are currently using \033[1m{target_upper}\033[0m as the target and \033[1m{scoring}\033[0m for scoring to predict prices for \033[1m{dataset_loc}\033[0m on \033[1m{dataset_date}\033[0m\n")
print(f"You are currently using these features for its prediction:\n\033[1m{pred_features}\033[0m\n")

if target in pred_features:
    print(f"WARNING: You are using \033[1m{target_upper}\033[0m as predictor. Please remove before proceeding.\n")

if target == 'price_binary':
    if 'price_class' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_class'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")
    if 'price_log' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_log'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")

if target == 'price_class':
    if 'price_binary' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_binary'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")
    if 'price_log' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_log'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")

if target == 'price_log':
    if 'price_class' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_class'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")
    if 'price_binary' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_binary'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")
        
if "occupancy_class" in pred_features and "occupancy_rate" in pred_features:
    print(f"WARNING: Please remove \033[1m'ocupancy_class'\033[0m or \033[1m'ocupancy_rate'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")

else:
    print("No issues with your selection of pred_features have been detected. Please make sure to manually check for correctness nevertheless.")
          

You are currently using PRICE_LOG as the target and neg_median_absolute_error for scoring to predict prices for berlin on 2020-03-17

You are currently using these features for its prediction:
['accommodates_per_bed', 'am_balcony', 'am_breakfast', 'am_child_friendly', 'am_elevator', 'am_essentials', 'am_pets_allowed', 'am_private_entrance', 'am_smoking_allowed', 'am_tv', 'bathrooms_log', 'bedrooms', 'calc_host_lst_count_sqrt_log', 'cancellation_policy', 'guests_included_calc', 'host_is_superhost', 'instant_bookable', 'maximum_nights', 'minimum_nights_sqrt', 'property_type', 'room_type', 'wk_mth_discount', 'zipcode']

No issues with your selection of pred_features have been detected. Please make sure to manually check for correctness nevertheless.


# Preprocessing (Train/Test Split and Pipeline)

In [93]:
# Drop columns
drop_columns = [el for el in data.columns if el not in pred_features]
drop_columns.remove(target)
data.drop(labels=drop_columns, inplace=True, axis=1)

In [94]:
# Drop rows (optional, just temporary)
#data = data[data.number_of_reviews_ltm_log>1.7]

## Preprocessing pipeline

In [95]:
# Create list for categorical predictors/features (used in "Scaling with Preprocessing Pipeline")
cat_features = list(data.columns[data.dtypes == object])
#cat_features.remove("neighbourhood")
#cat_features.remove("zipcode")
cat_features

['cancellation_policy', 'property_type', 'room_type', 'zipcode']

In [96]:
# Create list for numerical predictors/features (removing target column, used in "Scaling with Preprocessing Pipeline")
num_features = list(data.columns[data.dtypes != object])
num_features.remove(target)
num_features

['accommodates_per_bed',
 'am_balcony',
 'am_breakfast',
 'am_child_friendly',
 'am_elevator',
 'am_essentials',
 'am_pets_allowed',
 'am_private_entrance',
 'am_smoking_allowed',
 'am_tv',
 'bathrooms_log',
 'bedrooms',
 'calc_host_lst_count_sqrt_log',
 'guests_included_calc',
 'host_is_superhost',
 'instant_bookable',
 'maximum_nights',
 'minimum_nights_sqrt',
 'wk_mth_discount']

In [97]:
# Build preprocessor pipeline
# Pipeline for numerical features
num_pipeline = Pipeline([('imputer_num', SimpleImputer(strategy='median')),
                         ('std_scaler', StandardScaler())])

# Pipeline for categorical features
cat_pipeline = Pipeline([
    ('imputer_cat', SimpleImputer(strategy='constant', fill_value='missing')),
    ('1hot', OneHotEncoder(drop='first', handle_unknown='error'))
])

# Complete pipeline
preprocessor = ColumnTransformer([('num', num_pipeline, num_features),
                                  ('cat', cat_pipeline, cat_features)])

## Train/test split

In [98]:
# Define predictors and target variable
X = data.drop([target], axis=1)
y = data[target]

In [99]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    shuffle=True)
#                                                   stratify=y) # Use stratify=y if labels are inbalanced (e.g. most wines are 5 or 6; check with value_counts()!)

In [100]:
# Saving preprocessed X_train and X_test
X_train_prep_preprocessor = preprocessor.fit(X_train)

X_train_prep = X_train_prep_preprocessor.transform(X_train)
X_train_num_prep = num_pipeline.fit_transform(X_train[num_features])
X_test_prep = X_train_prep_preprocessor.transform(X_test)

In [101]:
# Get feature names from pipeline after one-hot encoding as "column_names"
onehot_columns = list(preprocessor.named_transformers_['cat']['1hot'].get_feature_names(cat_features))
column_names = num_features + onehot_columns

In [102]:
# Save preprocessor
save_model(X_train_prep_preprocessor, title="preprocessor_02", save="joblib")

In [103]:
# Save X_test
save_model(X_test, title="X_test_02", save="joblib")

## Global Functions and Variables

In [17]:
# Create "price_binary" at given split
price_binary = []
for price in data.price_log:
    if math.exp(price) <= binary_split:
        price_binary.append(1)
    else:
        price_binary.append(2)
data["price_binary"] = price_binary

In [18]:
# "mean_absolute_percentage_error": Function for mean absolute percentage error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [19]:
# "median_absolute_percentage_error": Function for median absolute percentage error (MAPE median)
def median_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

In [20]:
# "print_target_setting": Function for printing current setting for TARGET and the corresponding features
def print_target_setting():
    target_upper = target.upper()
    y_upper = y_train.name.upper()
    print(f"You are currently using \033[1m{target_upper}\033[0m as the target and \033[1m{scoring}\033[0m for scoring to predict prices for \033[1m{dataset_loc}\033[0m on \033[1m{dataset_date}\033[0m\n")
    print(f"The target variable y is set to \033[1m{y_upper}\033[0m\n")
    print(f"You are currently using these features for its prediction:\n\033[1m{pred_features}\033[0m\n")

In [21]:
# "get_feat_importances": Function for retrieving feature importances
def get_feat_importances(model, column_names=column_names):
    model=model
    feat_importances = pd.DataFrame(model.feature_importances_,
                 columns=['weight'],
                 index=column_names)
    feat_importances.sort_values('weight', inplace=True, ascending=False)
    return feat_importances

In [22]:
# "model_eval": Function for final evaluation of "best model"
def model_eval(y, y_pred, model="reg"):
    """
    Please always specify the type of model:
    Regression: model="reg"
    Binary Classification: model="bclf"
    Multiclass Classification: model="clf"
    """
    if model=="reg":
        print("MSE: {:.2f}".format(mean_squared_error(y, y_pred)))
        print("RMSE: {:.2f}".format(
        mean_squared_error(y, y_pred, squared=False)))
        print("MAE: {:.2f}".format(mean_absolute_error(y, y_pred)))
        print("R2: {:.2f}".format(r2_score(y, y_pred)))
        print("MAPE: {:.2f}".format(mean_absolute_percentage_error(y, y_pred)))
        print("MAPE median: {:.2f}".format(median_absolute_percentage_error(y, y_pred)))

    elif model=="bclf":
        print("Accuracy: {:.2f}".format(accuracy_score(y, y_pred)))
        print("Recall: {:.2f}".format(recall_score(y, y_pred)))
        print("Precision: {:.2f}".format(precision_score(y, y_pred)))
        print("F1 Score: {:.2f}".format(f1_score(y, y_pred)))
        print("ROC/AUC: {:.2f}".format(roc_auc_score(y, y_pred)))
        print("Confusion Matrix: \n" + str(confusion_matrix(y, y_pred)))

    elif model=="clf":
        print("Accuracy: {:.2f}".format(accuracy_score(y, y_pred)))
        print("Recall: {:.2f}".format(recall_score(y, y_pred, average='weighted')))
        print("Precision: {:.2f}".format(precision_score(y, y_pred, average='weighted')))
        print("F1 Score: {:.2f}".format(f1_score(y, y_pred, average='weighted')))
        print("Confusion Matrix: \n" + str(confusion_matrix(y, y_pred)))
    
    else:
        print("Please revise your parameters (e.g. provide a valid model).")

In [23]:
# "save_model": Function for saving model (pickle or joblib)
def save_model(model, title="unknown", save="joblib"):
    if save=="joblib":
        joblib.dump(model, f"saves/{dataset_loc}_{dataset_date}/{title}.pkl")
    elif save=="pickle":
        model.to_pickle(f"saves/{dataset_loc}_{dataset_date}/{title}.pkl")
#        save also evaluation (from y_test, y_train)

In [24]:
# "load_model": Function for loading model (pickle or joblib)
def load_model(title="unknown", dataset_loc=dataset_loc, dataset_date=dataset_date, load="joblib"):
    if load=="joblib":
        return joblib.load(f"saves/{dataset_loc}_{dataset_date}/{title}.pkl")

In [25]:
# "clf_learning_curves": Function to evaluate classification model based on learning curves
def clf_learning_curves(model):
# Fit model on training data
    model = model
    eval_set = [(X_train_prep, y_train), (X_test_prep, y_test)]
    model.fit(X_train_prep, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)

    # Make predictions for test data
    y_pred = model.predict(X_test_prep)
    predictions = [round(value) for value in y_pred]

    # Evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
    # Retrieve performance metrics
    results = model.evals_result()
    epochs = len(results['validation_0']['error'])
    x_axis = range(0, epochs)
    
    # Plot log loss
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
    ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
    ax.legend()
    pyplot.ylabel('Log Loss')
    pyplot.title('XGBoost Log Loss')
    pyplot.show()
    
    # Plot classification error
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0']['error'], label='Train')
    ax.plot(x_axis, results['validation_1']['error'], label='Test')
    ax.legend()
    pyplot.ylabel('Classification Error')
    pyplot.title('XGBoost Classification Error')
    pyplot.show()

# Modeling: Binary Classification ("price_binary")

## Apply Classification Models

In [277]:
# Print current setting for TARGET
print_target_setting()
print(f"Binary Split:  \033[1mUSD {binary_split}\033[0m")

You are currently using PRICE_BINARY as the target and roc_auc for scoring to predict prices for berlin on 2020-03-17

The target variable y is set to PRICE_BINARY

You are currently using these features for its prediction:
['accommodates', 'bathrooms_log', 'host_is_superhost', 'property_type', 'room_type', 'zipcode']

Binary Split:  USD 50


In [278]:
# Select models for comparison
bclfmodels = {
    'Baseline':
    DummyClassifier(strategy='most_frequent'),
    'LogReg':
    LogisticRegression(max_iter=1000),
    'KNN':
    KNeighborsClassifier(),
    'SVC':
    SVC(kernel='rbf', C=1E6),
    'Decision Tree':
    DecisionTreeClassifier(criterion="gini",
                           max_depth=3,
                           random_state=random_state),
    'Random Forest':
    RandomForestClassifier(random_state=random_state,
                           max_features='sqrt',
                           n_jobs=-1),
    'Gradient Boost':
    GradientBoostingClassifier(random_state=random_state),
    'XGBoost':
    XGBClassifier(),
    'AdaBoost':
    AdaBoostClassifier(random_state=random_state)
}

In [279]:
# Calculate and display results
results = pd.DataFrame(columns=['Model', 'Accuracy', 'Recall', 'Precision', 'F1 Score', 'ROC/AUC'])
i = 0
for m in bclfmodels.items():
    # Building a full pipeline with our preprocessor and a Classifier
    pipe = Pipeline([('preprocessor', preprocessor), (m[0], m[1])])
    # Making predictions on the training set using cross validation as well as calculating the probabilities
    y_train_pred = cross_val_predict(pipe,
                                     X_train,
                                     y_train.values.ravel(),
                                     cv=5,
                                     verbose=4,
                                     n_jobs=-1)
    # Calculating metrices
    temp = pd.DataFrame(
        {
            'Model': m[0],
            'Accuracy': accuracy_score(y_train, y_train_pred),
            'Recall': recall_score(y_train, y_train_pred, average="weighted"),
            'Precision': precision_score(
                y_train, y_train_pred, average="weighted"),
            'F1 Score': f1_score(y_train, y_train_pred, average="weighted"),
            'ROC/AUC': roc_auc_score(y_train, y_train_pred)
        },
        index=[i])
    print(f"Confusion Matrix {m[0]}: \n" +
          str(confusion_matrix(y_train, y_train_pred)))
    i += 1
    results = pd.concat([results, temp])
results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.6s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Confusion Matrix Baseline: 
[[   0 3993]
 [   0 4497]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Confusion Matrix LogReg: 
[[2941 1052]
 [ 793 3704]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.3s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

Confusion Matrix XGBoost: 
[[2860 1133]
 [ 667 3830]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.5s remaining:    0.8s


Confusion Matrix AdaBoost: 
[[2904 1089]
 [ 792 3705]]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


,Model,Accuracy,Recall,Precision,F1 Score,ROC/AUC
0,Baseline,0.529682,0.529682,0.280563,0.366825,0.500000
1,LogReg,0.782686,0.782686,0.782955,0.782085,0.780100
2,KNN,0.750530,0.750530,0.750375,0.750058,0.748286
3,SVC,0.763840,0.763840,0.763656,0.763571,0.762099
4,Decision Tree,0.774912,0.774912,0.777552,0.773200,0.770432
5,Random Forest,0.769847,0.769847,0.769864,0.769335,0.767489
6,Gradient Boost,0.789399,0.789399,0.791453,0.788104,0.785455
7,XGBoost,0.787986,0.787986,0.790136,0.786639,0.783966
8,AdaBoost,0.778445,0.778445,0.778896,0.777707,0.775578


## Clf Model 1: Logistic Regression

In [280]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_lr_bclf = Pipeline([('preprocessor', preprocessor),
                            ('lr_bclf',
                             LogisticRegression(penalty='l2',
                                                max_iter=100,
                                                C=0.9,
                                                random_state=random_state,
                                                l1_ratio=0.5,
                                                n_jobs=-1))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [281]:
# Display possible hyperparameters for LogisticRegression
test_lr_bclf = LogisticRegression()
test_lr_bclf.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

**Default values for LogisticRegression** (as base for hyperparameter search):

penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None,

In [283]:
# Define hyperparameter distribution
param_distribs_lr_bclf = {
    'lr_bclf__penalty': ['l1', 'l2'],
    'lr_bclf__max_iter': randint(low=10, high=100),
    'lr_bclf__C': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000],
    'lr_bclf__l1_ratio': [None, 0.1, 0.2, 0.3, 0.5, 0.9, 1]
}

In [284]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_lr_bclf = RandomizedSearchCV(pipeline_lr_bclf,
                                 param_distribs_lr_bclf,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=50,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_lr_bclf = rnd_lr_bclf.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.5s finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


In [285]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_lr_bclf.best_score_))
print("Best parameters:\n{}".format(rnd_lr_bclf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.85
Best parameters:
{'lr_bclf__C': 20, 'lr_bclf__l1_ratio': 0.1, 'lr_bclf__max_iter': 83, 'lr_bclf__penalty': 'l2'}


### Hyperparameter Tuning with GridSearchCV

In [289]:
# Define hyperparameter grid
param_grid_lr_bclf = {
    'lr_bclf__penalty': ['l1', 'l2'],
    'lr_bclf__max_iter': [100, 125, 150, 200],
    'lr_bclf__C': [10, 20, 30],
    'lr_bclf__l1_ratio': [None, 0.05, 0.1]
}

In [307]:
# Create and fit GridSearchCV, save "best_model"
grid_lr_bclf = GridSearchCV(pipeline_lr_bclf,
                            param_grid_lr_bclf,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_lr_bclf.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   18.9s finished


In [ ]:
# Save best model
best_model_lr_bclf = grid_lr_bclf.best_estimator_['lr_bclf']

In [291]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_lr_bclf.best_score_))
print("Best parameters:\n{}".format(grid_lr_bclf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.85
Best parameters:
{'lr_bclf__C': 20, 'lr_bclf__l1_ratio': None, 'lr_bclf__max_iter': 150, 'lr_bclf__penalty': 'l2'}


### Feature Importances

In [294]:
# Get and print feature importances
#grid_lr_bclf_fi = feat_importances(grid_lr_bclf, cv_model=True, named_steps='lr_bclf', column_names=column_names)
#grid_lr_bclf_fi

### !!! Model Evaluation

In [ ]:
# Predict target with "best model"
y_train_pred_lr_bclf = best_model_lr_bclf.predict(X_train_prep)

In [ ]:
# Final evaluation with "best model"
model_eval(y_train, y_train_pred_lr_bclf, model="bclf"):

In [567]:
# Calculating the accuracy, recall and precision for the test set with the best model
print("Accuracy: {:.2f}".format(accuracy_score(y_train, y_pred_lr_bclf)))
print("Recall: {:.2f}".format(recall_score(y_train, y_pred_lr_bclf)))
print("Precision: {:.2f}".format(precision_score(y_train, y_pred_lr_bclf)))
print("F1 Score: {:.2f}".format(f1_score(y_train, y_pred_lr_bclf)))
print("ROC/AUC: {:.2f}".format(roc_auc_score(y_train, y_pred_lr_bclf)))
print("Confusion Matrix: \n" + str(confusion_matrix(y_train, y_pred_lr_bclf)))

ValueError: Classification metrics can't handle a mix of continuous and binary targets

**Testing Set**

In [531]:
# Predict target with "best model"
y_test_pred_lr_bclf = best_model_lr_bclf.predict(X_test_prep)

In [564]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_lr_bclf, model="bclf")

MSE: 0.16
RMSE: 0.40
MAE: 0.31
R2: 0.50
MAPE: 7.97
MAPE median: 6.10


## !!! NN Model 1: TBD

## Best Model Evaluation with Testing Set and Export

In [ ]:
bclf_best_model = best_model_grid_lr_bclf

In [ ]:
# Export best model using joblib

In [ ]:
# Export best model at given binary_split
bclf_best_model.to_pickle(f"saves/{dataset_loc}_{dataset_date}/{target}_{binary_split}.pkl")

# Modeling: Multi-Class Classification ("price_class")

## Apply Classification Models

In [411]:
# Print current setting for TARGET
print_target_setting()

You are currently using PRICE_CLASS as the target and f1 for scoring to predict prices for berlin on 2020-03-17

The target variable y is set to PRICE_CLASS

You are currently using these features for its prediction:
['accommodates', 'accommodates_per_bed', 'am_balcony', 'am_breakfast', 'am_child_friendly', 'am_elevator', 'am_essentials', 'am_nature_and_views', 'am_pets_allowed', 'am_private_entrance', 'am_smoking_allowed', 'am_tv', 'am_white_goods', 'availability_90', 'bathrooms_log', 'bedrooms', 'calc_host_lst_count_sqrt_log', 'cancellation_policy', 'first_review_days_sqrt', 'host_acceptance_rate', 'host_is_superhost', 'host_response_rate', 'host_response_time', 'instant_bookable', 'last_review_days_sqrt', 'latitude', 'listing_no', 'longitude', 'maximum_nights', 'minimum_nights_log', 'neighbourhood_cleansed', 'number_of_reviews_ltm_log', 'price_extra_fees_sqrt', 'price_extra_people', 'property_type', 'review_scores_location', 'review_scores_rating_sqrt', 'room_type', 'text_len_sqrt',

In [412]:
# Select models for comparison
clfmodels = {
    'Baseline':
    DummyClassifier(strategy='most_frequent'),
    'LogReg':
    LogisticRegression(max_iter=1000),
    'KNN':
    KNeighborsClassifier(),
    'SVC':
    SVC(kernel='rbf', C=1E6),
    'Decision Tree':
    DecisionTreeClassifier(criterion="gini",
                           max_depth=3,
                           random_state=random_state),
    'Random Forest':
    RandomForestClassifier(random_state=random_state,
                           max_features='sqrt',
                           n_jobs=-1),
    'Gradient Boost':
    GradientBoostingClassifier(random_state=random_state),
    'XGBoost':
    XGBClassifier(),
    'AdaBoost':
    AdaBoostClassifier(random_state=random_state)
}

In [413]:
# Calculate and display results
results = pd.DataFrame(columns=['Model', 'Accuracy', 'Recall', 'Precision', 'F1 Score'])
i = 0
for m in clfmodels.items():
    # Building a full pipeline with our preprocessor and a Classifier
    pipe = Pipeline([('preprocessor', preprocessor), (m[0], m[1])])
    # Making predictions on the training set using cross validation as well as calculating the probabilities
    y_train_pred = cross_val_predict(pipe,
                                     X_train,
                                     y_train.values.ravel(),
                                     cv=5,
                                     verbose=4,
                                     n_jobs=-1)
    # Calculating metrices
    temp = pd.DataFrame(
        {
            'Model': m[0],
            'Accuracy': accuracy_score(y_train, y_train_pred),
            'Recall': recall_score(y_train, y_train_pred, average="weighted"),
            'Precision': precision_score(
                y_train, y_train_pred, average="weighted"),
            'F1 Score': f1_score(y_train, y_train_pred, average="weighted"),
        },
        index=[i])
    print(f"Confusion Matrix {m[0]}: \n" +
          str(confusion_matrix(y_train, y_train_pred)))
    i += 1
    results = pd.concat([results, temp])
results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.0s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Confusion Matrix Baseline: 
[[   0    0  307    0    0    0    0    0    0    0    0]
 [   0    0 1057    0    0    0    0    0    0    0    0]
 [   0    0 1353    0    0    0    0    0    0    0    0]
 [   0    0 1276    0    0    0    0    0    0    0    0]
 [   0    0  926    0    0    0    0    0    0    0    0]
 [   0    0  817    0    0    0    0    0    0    0    0]
 [   0    0  665    0    0    0    0    0    0    0    0]
 [   0    0  449    0    0    0    0    0    0    0    0]
 [   0    0  457    0    0    0    0    0    0    0    0]
 [   0    0  786    0    0    0    0    0    0    0    0]
 [   0    0  397    0    0    0    0    0    0    0    0]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.7s remaining:    4.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Confusion Matrix LogReg: 
[[ 55 125  78  21   4   4   3   0   1  11   5]
 [ 27 447 404 121  25   9   3   0   0  18   3]
 [ 12 347 497 301  81  61  22  10   1  18   3]
 [ 15 167 401 297 150 125  39  12  14  47   9]
 [  3  54 177 217 167 130  61  21  26  61   9]
 [  1  30  86 196 129 143  94  23  22  80  13]
 [  1  15  53 105  99 139  71  17  25 122  18]
 [  1   8  25  51  63  79  47  21  19 116  19]
 [  1   5  15  44  44  74  48  20  22 159  25]
 [  3   6  15  42  43  73  65  32  49 352 106]
 [  0   1   2   7  17   7   4   8   8 176 167]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.9s remaining:    4.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

Confusion Matrix XGBoost: 
[[ 66 134  76  14   3   6   0   0   0   6   2]
 [ 31 444 458  62  16  17   3   2   0  21   3]
 [  9 361 611 222  28  80   9   3   2  25   3]
 [  7 144 476 356  78 111  20   5   4  71   4]
 [  3  55 223 255 104 152  27   8  13  80   6]
 [  0  24 117 214  78 208  36  10  20  98  12]
 [  0  20  68 156  57 139  51  11  19 125  19]
 [  0   7  25  83  34  86  23  22  11 142  16]
 [  1   9  23  63  26  75  18   8  16 189  29]
 [  0   4  17  72  40  72  29   8  20 421 103]
 [  0   1   4  11   7  10   4   3   5 157 195]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.0s remaining:    3.0s


Confusion Matrix AdaBoost: 
[[ 55  99 103  22   2   4   1   0   0  14   7]
 [130 314 443 108  11  14   1   1   0  25  10]
 [ 98 258 571 249  37  65  18   3   4  34  16]
 [ 55 139 406 313  78 138  24   8  11  91  13]
 [ 26  63 182 250  85 155  20   5   6 107  27]
 [ 15  33 112 214  64 179  37  10  15 108  30]
 [  6  23  80 137  37 123  35   6   5 170  43]
 [  3  10  31  74  32  97  26   6   9 136  25]
 [  5  15  25  54  27  74  19   3   9 180  46]
 [  7  30  13  57  25 100  30  10  14 348 152]
 [  0   5   8  10   3  12  10   2   7 171 169]]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.0s finished


,Model,Accuracy,Recall,Precision,F1 Score
0,Baseline,0.159364,0.159364,0.025397,0.043812
1,LogReg,0.263722,0.263722,0.252461,0.251175
2,KNN,0.225088,0.225088,0.229258,0.218664
3,SVC,0.284217,0.284217,0.282050,0.281744
4,Decision Tree,0.239458,0.239458,0.260865,0.163596
5,Random Forest,0.301649,0.301649,0.307399,0.286645
6,Gradient Boost,0.289635,0.289635,0.276851,0.274505
7,XGBoost,0.293757,0.293757,0.286248,0.270193
8,AdaBoost,0.245465,0.245465,0.225560,0.221782


In [415]:
# Apply LogReg with OVR
lr_ovr_model = LogisticRegression(multi_class='ovr', max_iter=200)
# fit model
lr_ovr_model.fit(X_train_prep, y_train)
# make predictions
y_pred_lr_ovr = lr_ovr_model.predict(X_train_prep)

print('Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Recall: ', recall_score(y_train, y_train_pred, average="weighted"))
print('Precision: ', precision_score(y_train, y_train_pred, average="weighted"))
print('F1 Score: ', f1_score(y_train, y_train_pred, average="weighted"))

Accuracy:  0.24546525323910484
Recall:  0.24546525323910484
Precision:  0.22555966733807414
F1 Score:  0.22178198870408516


## Clf Model 1: Logistic Regression

In [426]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_lr_clf = Pipeline([('preprocessor', preprocessor),
                            ('lr_clf',
                             LogisticRegression(penalty='l2',
                                                max_iter=100,
                                                C=0.9,
#                                                multi_class='multinomial',
                                                random_state=random_state,
                                                l1_ratio=0.5,
                                                n_jobs=-1))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [427]:
# Display possible hyperparameters for LogisticRegression
test_lr_clf = LogisticRegression()
test_lr_clf.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

**Default values for LogisticRegression** (as base for hyperparameter search):

penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None,

In [428]:
# Define hyperparameter distribution
param_distribs_lr_clf = {
    'lr_clf__penalty': ['l1', 'l2'],
    'lr_clf__max_iter': randint(low=10, high=100),
    'lr_clf__C': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000],
    'lr_clf__l1_ratio': [None, 0.1, 0.2, 0.3, 0.5, 0.9, 1]
}

In [429]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_lr_clf = RandomizedSearchCV(pipeline_lr_clf,
                                 param_distribs_lr_clf,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=50,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_lr_clf = rnd_lr_clf.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   35.5s finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


In [430]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_lr_clf.best_score_))
print("Best parameters:\n{}".format(rnd_lr_clf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.25
Best parameters:
{'lr_clf__C': 1, 'lr_clf__l1_ratio': 0.9, 'lr_clf__max_iter': 96, 'lr_clf__penalty': 'l2'}


### Hyperparameter Tuning with GridSearchCV

In [434]:
# Define hyperparameter grid
param_grid_lr_clf = {
#    'lr_clf__penalty': ['l1', 'l2'],
    'lr_clf__max_iter': [85, 100, 125],
    'lr_clf__C': [0.1, 0.5, 1, 2],
    'lr_clf__l1_ratio': [0.8, 0.9, 1]
}

In [435]:
# Create and fit GridSearchCV, save "best_model"
grid_lr_clf = GridSearchCV(pipeline_lr_clf,
                            param_grid_lr_clf,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_lr_clf.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.2min finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['accommodates',
                                                                          'accommodates_per_bed',
                                                                          'am_balcony',
                                                                          'am_breakfast',
                                  

In [436]:
# Save best model
best_model_lr_clf = grid_lr_clf.best_estimator_['lr_clf']

In [437]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_lr_clf.best_score_))
print("Best parameters:\n{}".format(grid_lr_clf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.25
Best parameters:
{'lr_clf__C': 1, 'lr_clf__l1_ratio': 0.8, 'lr_clf__max_iter': 100}


### Feature Importances

### Model Evaluation

**Training Set**

In [ ]:
# Predict target with "best model"
y_train_pred_lr_clf = best_model_lr_clf.predict(X_train_prep)

In [ ]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_lr_clf, model="clf")

In [442]:
# Calculating the accuracy, recall and precision for the test set with the best model
print("Accuracy: {:.2f}".format(accuracy_score(y_train, y_pred_lr_clf)))
print("Recall: {:.2f}".format(recall_score(y_train, y_pred_lr_clf, average='weighted')))
print("Precision: {:.2f}".format(precision_score(y_train, y_pred_lr_clf, average='weighted')))
print("F1 Score: {:.2f}".format(f1_score(y_train, y_pred_lr_clf, average='weighted')))
#print("ROC/AUC: {:.2f}".format(roc_auc_score(y_train, y_pred_lr_clf, multi_class='ovr')))
print("Confusion Matrix: \n" + str(confusion_matrix(y_train, y_pred_lr_clf)))

Accuracy: 0.35
Recall: 0.35
Precision: 0.35
F1 Score: 0.34
Confusion Matrix: 
[[ 74 120  73  19   4   3   3   0   0   7   4]
 [ 20 543 350  94  20   7   3   1   1  18   0]
 [ 12 299 648 225  63  60  19   5   1  18   3]
 [ 11 144 369 412 112 119  32  11   9  48   9]
 [  6  45 178 211 209 127  52  17  13  62   6]
 [  0  27  75 182 112 228  67  20  15  82   9]
 [  1  10  58  97  84 125 126  15  21 111  17]
 [  0   7  23  51  54  70  48  42  18 118  18]
 [  1   8  15  47  42  64  39  13  56 149  23]
 [  2   2  15  35  38  68  49  23  33 450  71]
 [  0   1   3   6  14   9   8   6   7 139 204]]


**Testing Set**

In [531]:
# Predict target with "best model"
y_test_pred_lr_clf = best_model_lr_clf.predict(X_test_prep)

In [564]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_lr_clf, model="clf")

MSE: 0.16
RMSE: 0.40
MAE: 0.31
R2: 0.50
MAPE: 7.97
MAPE median: 6.10


## Clf Model 2: Random Forest

In [443]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_rf_clf = Pipeline([('preprocessor', preprocessor),
                            ('rf_clf',
                             RandomForestClassifier(n_estimators=110,
                                                    random_state=random_state,
                                                    max_depth=5,
                                                    max_features=20,
                                                    n_jobs=-1))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [444]:
# Display possible hyperparameters for RandomForestClassifier
test_rf_clf = RandomForestClassifier()
test_rf_clf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

**Default values for LogisticRegression** (as base for hyperparameter search):

penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None,

In [445]:
# Define hyperparameter distribution
param_distribs_rf_clf = {
    'rf_clf__n_estimators': randint(low=10, high=500),
    'rf_clf__max_depth': randint(low=1, high=30),
    'rf_clf__max_features': randint(low=1, high=100),
}

In [446]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_rf_clf = RandomizedSearchCV(pipeline_rf_clf,
                           param_distribs_rf_clf,
                           cv=5,
                           scoring=scoring,
                           return_train_score=True,
                           verbose=4,
                           n_jobs=-1,
                         random_state=random_state)

rnd_rf_clf.fit(X_train, y_train)
best_model_rnd_rf_clf = rnd_rf_clf.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  9.0min finished


In [450]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_rf_clf.best_score_))
print("Best parameters:\n{}".format(rnd_rf_clf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.30
Best parameters:
{'rf_clf__max_depth': 21, 'rf_clf__max_features': 33, 'rf_clf__n_estimators': 469}


### Hyperparameter Tuning with GridSearchCV

In [454]:
# Define hyperparameter grid
param_grid_rf_clf = {
    'rf_clf__n_estimators': [450, 500],
    'rf_clf__max_depth': [20, 25],
    'rf_clf__max_features': [30, 35],
}

In [455]:
# Create and fit GridSearchCV, save "best_model"
grid_rf_clf = GridSearchCV(pipeline_rf_clf,
                            param_grid_rf_clf,
                            cv=4,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_rf_clf.fit(X_train, y_train)

Fitting 4 folds for each of 8 candidates, totalling 32 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed: 10.1min finished


In [456]:
# Save best model
best_model_rf_clf = grid_rf_clf.best_estimator_['rf_clf']

In [457]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_rf_clf.best_score_))
print("Best parameters:\n{}".format(grid_rf_clf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.30
Best parameters:
{'rf_clf__max_depth': 25, 'rf_clf__max_features': 35, 'rf_clf__n_estimators': 450}


### Feature Importances

In [476]:
# Get and print feature importances
grid_rf_clf_fi = feat_importances(grid_rf_clf, cv_model=True, named_steps='rf_clf', column_names=column_names)
grid_rf_clf_fi

,weight
latitude,0.053475
longitude,0.052189
text_len_sqrt,0.049861
listing_no,0.046906
first_review_days_sqrt,0.045590
last_review_days_sqrt,0.045221
availability_90,0.043774
price_extra_fees_sqrt,0.041142
number_of_reviews_ltm_log,0.039622
host_acceptance_rate,0.034078


### Model Evaluation

**Training Set**

In [ ]:
# Predict target with "best model"
y_train_pred_rf_clf = best_model_rf_clf.predict(X_train_prep)

In [459]:
# Final evaluation with "best model"
model_eval(y_train, y_train_pred_rf_clf, model="clf")

print("Accuracy: {:.2f}".format(accuracy_score(y_train, y_pred_rf_clf)))
print("Recall: {:.2f}".format(recall_score(y_train, y_pred_rf_clf, average='weighted')))
print("Precision: {:.2f}".format(precision_score(y_train, y_pred_rf_clf, average='weighted')))
print("F1 Score: {:.2f}".format(f1_score(y_train, y_pred_rf_clf, average='weighted')))
#print("ROC/AUC: {:.2f}".format(roc_auc_score(y_train, y_pred_lr_clf, multi_class='ovr')))
print("Confusion Matrix: \n" + str(confusion_matrix(y_train, y_pred_rf_clf)))

Accuracy: 1.00
Recall: 1.00
Precision: 1.00
F1 Score: 1.00
Confusion Matrix: 
[[ 307    0    0    0    0    0    0    0    0    0    0]
 [   0 1057    0    0    0    0    0    0    0    0    0]
 [   0    0 1353    0    0    0    0    0    0    0    0]
 [   0    0    0 1276    0    0    0    0    0    0    0]
 [   0    0    0    0  926    0    0    0    0    0    0]
 [   0    0    0    0    0  817    0    0    0    0    0]
 [   0    0    0    0    0    0  665    0    0    0    0]
 [   0    0    0    0    0    0    0  449    0    0    0]
 [   0    0    0    0    0    0    0    0  457    0    0]
 [   0    0    0    0    0    0    0    0    0  786    0]
 [   0    0    0    0    0    0    0    0    0    0  397]]


**Testing Set**

In [531]:
# Predict target with "best model"
y_test_pred_rf_clf = best_model_rf_clf.predict(X_test_prep)

In [564]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_rf_clf, model="clf")

MSE: 0.16
RMSE: 0.40
MAE: 0.31
R2: 0.50
MAPE: 7.97
MAPE median: 6.10


## Clf Model 3: XGBoost

In [460]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_xgb_clf = Pipeline([('preprocessor', preprocessor),
                             ('xgb_clf',
                              XGBClassifier(n_estimators=110,
                                            random_state=random_state,
                                            max_depth=5,
                                            max_features=20,
                                            scoring=scoring,
                                            n_jobs=-1))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [461]:
# Display possible hyperparameters for LogisticRegression
test_xgb_clf = XGBClassifier()
test_xgb_clf.get_params().keys()

dict_keys(['base_score', 'booster', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'gamma', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'n_estimators', 'n_jobs', 'nthread', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'seed', 'silent', 'subsample', 'verbosity'])

**Default values for LogisticRegression** (as base for hyperparameter search):

penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None,

In [462]:
# Define hyperparameter distribution
param_distribs_xgb_clf = {
    'xgb_clf__n_estimators': randint(low=10, high=200),
    'xgb_clf__max_depth': randint(low=1, high=10),
    'xgb_clf__learning_rate': [0.05, 0.1, 0.2, 0.3]
}

In [463]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_xgb_clf = RandomizedSearchCV(pipeline_xgb_clf,
                                 param_distribs_xgb_clf,
                                 cv=5,
                                 n_iter=20,
                                 scoring=scoring,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_xgb_clf = rnd_xgb_clf.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 20.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 25.0min finished


In [464]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_xgb_clf.best_score_))
print("Best parameters:\n{}".format(rnd_xgb_clf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.30
Best parameters:
{'xgb_clf__learning_rate': 0.2, 'xgb_clf__max_depth': 8, 'xgb_clf__n_estimators': 126}


### Hyperparameter Tuning with GridSearchCV

In [466]:
# Define hyperparameter grid
param_grid_xgb_clf = {
    'xgb_clf__n_estimators': [120, 130],
    'xgb_clf__max_depth': [8, 10],
    'xgb_clf__learning_rate': [0.2, 0.3]
}

In [467]:
# Create and fit GridSearchCV, save "best_model"
grid_xgb_clf = GridSearchCV(pipeline_xgb_clf,
                            param_grid_xgb_clf,
                            cv=3,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_xgb_clf.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  7.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  7.4min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['accommodates',
                                                                          'accommodates_per_bed',
                                                                          'am_balcony',
                                                                          'am_breakfast',
                                  

In [469]:
# Save best model
best_model_xgb_clf = grid_xgb_clf.best_estimator_['xgb_clf']

In [470]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_xgb_clf.best_score_))
print("Best parameters:\n{}".format(grid_xgb_clf.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
0.29
Best parameters:
{'xgb_clf__learning_rate': 0.2, 'xgb_clf__max_depth': 10, 'xgb_clf__n_estimators': 130}


### Feature Importances

In [475]:
# Get and print feature importances
grid_xgb_clf_fi = feat_importances(grid_xgb_clf, cv_model=True, named_steps='xgb_clf', column_names=column_names)
grid_xgb_clf_fi

,weight
room_type_Private room,0.054966
room_type_Shared room,0.018998
property_type_Boutique hotel,0.017493
zipcode_zip_13407,0.013383
bedrooms,0.013143
neighbourhood_cleansed_Ost 2,0.012749
neighbourhood_cleansed_Alt-Lichtenberg,0.012494
room_type_Hotel room,0.011030
zipcode_zip_13187,0.010751
zipcode_zip_14057,0.010312


### Model Evaluation

**Training Set**

In [ ]:
# Predict target with "best model"
y_train_pred_xgb_clf = best_model_xgb_clf.predict(X_train_prep)

In [477]:
# Final evaluation with "best model"
model_eval(y_train, y_train_pred_xgb_clf, model="clf")

Accuracy: 1.00
Recall: 1.00
Precision: 1.00
F1 Score: 1.00
Confusion Matrix: 
[[ 307    0    0    0    0    0    0    0    0    0    0]
 [   0 1057    0    0    0    0    0    0    0    0    0]
 [   0    0 1353    0    0    0    0    0    0    0    0]
 [   0    0    0 1276    0    0    0    0    0    0    0]
 [   0    0    0    0  926    0    0    0    0    0    0]
 [   0    0    0    0    0  817    0    0    0    0    0]
 [   0    0    0    0    0    0  665    0    0    0    0]
 [   0    0    0    0    0    0    0  449    0    0    0]
 [   0    0    0    0    0    0    0    0  457    0    0]
 [   0    0    0    0    0    0    0    0    0  786    0]
 [   0    0    0    0    0    0    0    0    0    0  397]]


**Testing Set**

In [531]:
# Predict target with "best model"
y_test_pred_xgb_clf = best_model_xgb_clf.predict(X_test_prep)

In [564]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_xgb_clf, model="clf")

MSE: 0.16
RMSE: 0.40
MAE: 0.31
R2: 0.50
MAPE: 7.97
MAPE median: 6.10


## !!! NN Model 1: TBD

# Modeling: Regression ("price_log")

## Apply Regression Models

In [26]:
# Print current setting for TARGET
print_target_setting()

You are currently using PRICE_LOG as the target and neg_median_absolute_error for scoring to predict prices for berlin on 2020-03-17

The target variable y is set to PRICE_LOG

You are currently using these features for its prediction:
['accommodates_per_bed', 'am_balcony', 'am_breakfast', 'am_child_friendly', 'am_elevator', 'am_essentials', 'am_pets_allowed', 'am_private_entrance', 'am_smoking_allowed', 'am_tv', 'bathrooms_log', 'bedrooms', 'calc_host_lst_count_sqrt_log', 'cancellation_policy', 'guests_included_calc', 'host_is_superhost', 'instant_bookable', 'maximum_nights', 'minimum_nights_log', 'property_type', 'room_type', 'wk_mth_discount', 'zipcode']



In [27]:
# Select models for comparison
regmodels = {
    'Baseline':
    DummyRegressor(strategy='mean'),
    'LinReg':
    LinearRegression(),
    'Passive Aggressive':
    PassiveAggressiveRegressor(),
    #        'RANSAC' : RANSACRegressor(),
    'ElasticNet':
    ElasticNet(),
    'Stochastic Gradient Descent':
    SGDRegressor(max_iter=1000, tol=1e-3),
    'Decision Tree':
    DecisionTreeRegressor(criterion="mse",
                          max_depth=3,
                          random_state=random_state),
    'Random Forest':
    RandomForestRegressor(random_state=random_state,
                          max_features='sqrt',
                          n_jobs=-1),
    'Gradient Boost':
    GradientBoostingRegressor(random_state=random_state),
    'XGBoost':
    XGBRegressor(),
    'AdaBoost':
    AdaBoostRegressor(random_state=random_state),
    'SVR':
    SVR(),
    'CatBoost':
    CatBoostRegressor()
}

In [28]:
# Calculate and display results
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'R2', 'MAE', 'MAPE', 'MAPE median'])
i = 0
for m in regmodels.items():
    # Building a full pipeline with our preprocessor and a Classifier
    pipe = Pipeline([('preprocessor', preprocessor), (m[0], m[1])])
    # Making predictions on the training set using cross validation as well as calculating the probabilities
    y_train_pred = cross_val_predict(pipe,
                                     X_train,
                                     y_train.values.ravel(),
                                     cv=5,
                                     verbose=4,
                                     n_jobs=-1)
    # Calculating metrices
    temp = pd.DataFrame(
        {
            'Model':
            m[0],
            'MSE':
            "{:.2f}".format(mean_squared_error(y_train, y_train_pred)),
            'RMSE':
            "{:.2f}".format(
                mean_squared_error(y_train, y_train_pred, squared=False)),
            'R2':
            "{:.2f}".format(r2_score(y_train, y_train_pred)),
            'MAE':
            "{:.2f}".format(mean_absolute_error(y_train, y_train_pred)),
            'MAPE': mean_absolute_percentage_error(y_train, y_train_pred),
            'MAPE median': median_absolute_percentage_error(y_train, y_train_pred)
        },
        index=[i])
    i += 1
    results = pd.concat([results, temp])
results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.0s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_

,Model,MSE,RMSE,R2,MAE,MAPE,MAPE median
0,Baseline,0.35,0.59,-0.00,0.47,11.963293,10.312244
1,LinReg,0.13,0.37,0.61,0.28,7.099945,5.622496
2,Passive Aggressive,0.25,0.50,0.29,0.38,9.588548,7.757167
3,ElasticNet,0.35,0.59,-0.00,0.47,11.963293,10.312244
4,Stochastic Gradient Descent,0.47,0.69,-0.36,0.51,12.587933,9.711597
5,Decision Tree,0.19,0.43,0.46,0.33,8.384422,6.747336
6,Random Forest,0.12,0.35,0.64,0.27,6.849030,5.352247
7,Gradient Boost,0.13,0.36,0.63,0.28,7.033132,5.646463
8,XGBoost,0.13,0.36,0.63,0.28,7.026799,5.606353
9,AdaBoost,0.16,0.41,0.52,0.32,8.001659,6.540991


## Reg Model 1: XGBoost

In [29]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_xgb_reg = Pipeline([('preprocessor', preprocessor),
                             ('xgb_reg',
                              XGBRegressor(n_estimators=182,
                                           learning_rate=0.45,
                                           random_state=random_state,
                                           max_depth=5,
                                           gamma=0.3,
                                           bootstrap=True,
                                           max_features=21,
                                           scoring=scoring,
                                           n_jobs=-1))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [859]:
# Display possible hyperparameters for XGBoost Regressor
test_xgb_reg = XGBRegressor()
test_xgb_reg.get_params().keys()

dict_keys(['base_score', 'booster', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'gamma', 'importance_type', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'n_estimators', 'n_jobs', 'nthread', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'seed', 'silent', 'subsample', 'verbosity'])

**Default values for XGBRegressor** (as base for hyperparameter search):

max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective='reg:linear', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, importance_type='gain'

In [49]:
# Define hyperparameter distribution
param_distribs_xgb_reg = {
    'xgb_reg__n_estimators': randint(low=80, high=300),
    'xgb_reg__bootstrap': [True, False],
    'xgb_reg__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'xgb_reg__max_depth': randint(low=1, high=7),
    'xgb_reg__max_features': randint(low=1, high=40),
    'xgb_reg__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
}

In [50]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_xgb_reg = RandomizedSearchCV(pipeline_xgb_reg,
                                 param_distribs_xgb_reg,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=10,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_xgb_reg = rnd_xgb_reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.0min finished


[12:04:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [51]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_xgb_reg.best_score_))
print("Best parameters:\n{}".format(rnd_xgb_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_xgb_reg.best_estimator_))

Best score:
-0.21
Best parameters:
{'xgb_reg__bootstrap': True, 'xgb_reg__gamma': 0.3, 'xgb_reg__learning_rate': 0.45, 'xgb_reg__max_features': 8, 'xgb_reg__n_estimators': 268}


### Hyperparameter Tuning with GridSearchCV

In [30]:
# Define hyperparameter grid
param_grid_xgb_reg = {
#    'xgb_reg__bootstrap': [True, False],
#    'xgb_reg__n_estimators': [190, 230, 290],
#    'xgb_reg__max_features': [40, 45],
    'xgb_reg__max_depth': [4, 5],
    'xgb_reg__learning_rate': [0.42, 0.45, 0.48]
}

In [31]:
# Create and fit GridSearchCV, save "best_model"
grid_xgb_reg = GridSearchCV(pipeline_xgb_reg,
                            param_grid_xgb_reg,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_xgb_reg = grid_xgb_reg.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    9.2s finished
C:\Users\Mauricio\anaconda3\envs\airbnb42\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[20:51:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [32]:
# Save best model
best_model_xgb_reg = grid_xgb_reg.best_estimator_['xgb_reg']

In [33]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_xgb_reg.best_score_))
print("Best parameters:\n{}".format(grid_xgb_reg.best_params_))
#print("Best estimator:\n{}".format(grid_xgb_reg.best_estimator_))

Best score:
-0.22
Best parameters:
{'xgb_reg__learning_rate': 0.42, 'xgb_reg__max_depth': 5}


### Feature Importances

In [34]:
# Display feature importances
fi_xgb_reg = get_feat_importances(best_model_xgb_reg)
fi_xgb_reg

,weight
room_type_Private room,0.354507
bedrooms,0.049805
room_type_Shared room,0.044553
guests_included_calc,0.035743
property_type_Boutique hotel,0.032164
room_type_Hotel room,0.030424
zipcode_zip_10119,0.020782
zipcode_zip_10117,0.017576
am_elevator,0.017562
bathrooms_log,0.015473


### Final Evaluation Best Model

In [35]:
# Load existing model
#load_best_model = load_model(title="best_model_xgb_reg", dataset_loc=dataset_loc, dataset_date=dataset_date)
#load_best_cv = load_model(title="best_cv_xgb_reg", dataset_loc=dataset_loc, dataset_date=dataset_date)

**Learning Curves (Overfitting)**

**Training Set**

In [36]:
# Predict target with "best model"
y_train_pred_xgb_reg = best_model_xgb_reg.predict(X_train_prep)

In [37]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_xgb_reg, model="reg")

MSE: 0.09
RMSE: 0.30
MAE: 0.23
R2: 0.74
MAPE: 5.81
MAPE median: 4.56


**Testing Set**

In [38]:
# Predict target with "best model"
y_test_pred_xgb_reg = best_model_xgb_reg.predict(X_test_prep)

In [39]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_xgb_reg, model="reg")

MSE: 0.13
RMSE: 0.36
MAE: 0.27
R2: 0.63
MAPE: 6.78
MAPE median: 5.18


In [40]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_xgb_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.34298203, 0.37207619])

**Median Price Intervals**

In [41]:
# Save MAPE_median as variable
MAPE_median_xgb_reg = (median_absolute_percentage_error(y_test, y_test_pred_xgb_reg))/100

In [42]:
# Calculate price interval for MAPE median
y_pred_interval_xgb_reg = tuple([(round(math.exp(el-el*MAPE_median_xgb_reg),2),round(math.exp(el+el*MAPE_median_xgb_reg),2)) for el in y_test_pred_xgb_reg])
y_pred_interval_xgb_reg

((67.34, 106.66),
 (56.01, 86.95),
 (41.73, 62.72),
 (29.38, 42.5),
 (56.9, 88.48),
 (33.12, 48.55),
 (37.71, 56.06),
 (72.91, 116.48),
 (92.31, 151.33),
 (39.06, 58.3),
 (61.6, 96.61),
 (44.24, 66.93),
 (35.15, 51.85),
 (138.36, 237.07),
 (34.38, 50.6),
 (44.02, 66.56),
 (40.47, 60.63),
 (123.6, 209.19),
 (26.51, 37.92),
 (62.74, 98.59),
 (66.59, 105.34),
 (64.94, 102.45),
 (24.13, 34.16),
 (71.73, 114.39),
 (45.55, 69.13),
 (41.77, 62.79),
 (72.45, 115.67),
 (76.33, 122.56),
 (69.91, 111.17),
 (46.95, 71.5),
 (101.6, 168.31),
 (33.31, 48.86),
 (35.2, 51.93),
 (30.72, 44.65),
 (55.24, 85.61),
 (131.94, 224.89),
 (34.53, 50.84),
 (41.61, 62.52),
 (26.27, 37.54),
 (34.61, 50.96),
 (48.91, 74.81),
 (40.71, 61.02),
 (56.73, 88.18),
 (23.73, 33.53),
 (74.45, 119.21),
 (50.95, 78.28),
 (128.08, 217.61),
 (110.23, 184.23),
 (24.41, 34.6),
 (34.05, 50.05),
 (50.35, 77.25),
 (59.33, 92.68),
 (83.86, 136.04),
 (68.16, 108.1),
 (17.73, 24.28),
 (52.49, 80.9),
 (115.39, 193.83),
 (37.34, 55.46),


**Save Model and Params**

In [43]:
# Save best model and cv
save_model(best_model_xgb_reg, title="best_model_xgb_reg_02", save="joblib")
save_model(grid_xgb_reg, title="best_cv_xgb_reg_02", save="joblib")

## Reg Model 2: Support Vector Machines

In [49]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_svm_reg = Pipeline([('preprocessor', preprocessor),
                             ('svm_reg',
                              SVR(kernel='rbf',
                                  C=50,
                                  degree=4,
                                  gamma=0.005,
                                  epsilon=0.3))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [45]:
# Display possible hyperparameters for Support Vector Machine
test_svr_reg = SVR()
test_svr_reg.get_params().keys()

dict_keys(['C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', 'tol', 'verbose'])

**Default values for Support Vector Machine** (as base for hyperparameter search):

kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1

In [46]:
# Define hyperparameter distribution
param_distribs_svm_reg = {
#    'svm_reg__kernel': ['linear', 'poly', 'rbf'],
    'svm_reg__C': [0.1, 0.5, 0.8, 1, 1.5, 2, 3, 5, 10, 50, 100],        # initial: [0.1, 0.5, 1, 2, 5, 10, 50, 100, 500, 1000]
    'svm_reg__gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1],   # initial: [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1]
    'svm_reg__epsilon': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.9],            # initial: [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.9]
    'svm_reg__degree': randint(low=1, high=5)
}

In [47]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_svm_reg = RandomizedSearchCV(pipeline_svm_reg,
                                 param_distribs_svm_reg,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=10,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_svm_reg = rnd_svm_reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.3min finished


In [48]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_svm_reg.best_score_))
print("Best parameters:\n{}".format(rnd_svm_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_svm_reg.best_estimator_))

Best score:
-0.22
Best parameters:
{'svm_reg__C': 50, 'svm_reg__degree': 4, 'svm_reg__epsilon': 0.3, 'svm_reg__gamma': 0.005}


### Hyperparameter Tuning with GridSearchCV

In [50]:
# Define hyperparameter grid
param_grid_svm_reg = {
#    'svm_reg__kernel': ['linear', 'poly', 'rbf'],
    'svm_reg__gamma': [0.003, 0.005, 0.007],
    'svm_reg__C': [40, 50, 60],
    'svm_reg__degree': [3, 4, 5]
}

In [51]:
# Create and fit GridSearchCV, save "best_model"
grid_svm_reg = GridSearchCV(pipeline_svm_reg,
                            param_grid_svm_reg,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_svm_reg = grid_svm_reg.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.9min finished


In [52]:
# Save best model
best_model_svm_reg = grid_svm_reg.best_estimator_["svm_reg"]

In [53]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_svm_reg.best_score_))
print("Best parameters:\n{}".format(grid_svm_reg.best_params_))
#print("Best estimator:\n{}".format(grid_svm_reg.best_estimator_))

Best score:
-0.22
Best parameters:
{'svm_reg__C': 50, 'svm_reg__degree': 3, 'svm_reg__gamma': 0.005}


### Feature Importances

In [54]:
# Display feature importances
#fi_svm_reg = get_feat_importances(best_model_svm_reg)
#fi_svm_reg

### Final Evaluation Best Model

In [55]:
# Load existing model
#load_best_model = load_model(title="best_model_svm_reg_01", dataset_loc=dataset_loc, dataset_date=dataset_date)
#load_best_cv = load_model(title="best_cv_svm_reg_01", dataset_loc=dataset_loc, dataset_date=dataset_date)

**Learning Curves (Overfitting)**

**Training Set**

In [56]:
# Predict target with "best model"
y_train_pred_svm_reg = best_model_svm_reg.predict(X_train_prep)

In [57]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_svm_reg, model="reg")

MSE: 0.09
RMSE: 0.31
MAE: 0.24
R2: 0.73
MAPE: 6.10
MAPE median: 5.43


**Testing Set**

In [58]:
# Predict target with "best model"
y_test_pred_svm_reg = best_model_svm_reg.predict(X_test_prep)

In [59]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_svm_reg, model="reg")

MSE: 0.13
RMSE: 0.36
MAE: 0.27
R2: 0.64
MAPE: 6.81
MAPE median: 5.33


In [60]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_svm_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.34191936, 0.36906891])

**Median Price Intervals**

In [61]:
# Save MAPE_median as variable
MAPE_median_svm_reg = (median_absolute_percentage_error(y_test, y_test_pred_svm_reg))/100

In [62]:
# Calculate price interval for MAPE median
y_pred_interval_svm_reg = tuple([(round(math.exp(el-el*MAPE_median_svm_reg),2),round(math.exp(el+el*MAPE_median_svm_reg),2)) for el in y_test_pred_svm_reg])
y_pred_interval_svm_reg

((70.78, 114.35),
 (53.15, 83.13),
 (40.03, 60.64),
 (32.39, 47.92),
 (54.4, 85.32),
 (35.24, 52.63),
 (40.94, 62.19),
 (89.25, 148.0),
 (108.69, 184.27),
 (37.11, 55.74),
 (51.95, 81.05),
 (53.21, 83.25),
 (36.17, 54.18),
 (120.27, 206.23),
 (35.64, 53.29),
 (62.12, 98.89),
 (34.2, 50.9),
 (112.62, 191.7),
 (24.49, 35.11),
 (64.86, 103.76),
 (82.26, 135.16),
 (51.58, 80.41),
 (27.05, 39.21),
 (63.41, 101.18),
 (36.54, 54.79),
 (40.69, 61.75),
 (77.7, 126.85),
 (83.29, 137.05),
 (67.83, 109.05),
 (43.07, 65.79),
 (74.43, 120.93),
 (32.47, 48.05),
 (35.69, 53.38),
 (31.79, 46.93),
 (56.51, 89.01),
 (143.54, 251.1),
 (31.36, 46.22),
 (39.16, 59.18),
 (24.67, 35.4),
 (33.29, 49.41),
 (43.14, 65.91),
 (41.17, 62.57),
 (48.8, 75.6),
 (25.84, 37.27),
 (82.56, 135.7),
 (53.87, 84.39),
 (115.69, 197.51),
 (103.1, 173.75),
 (22.71, 32.28),
 (35.07, 52.35),
 (49.66, 77.08),
 (45.35, 69.68),
 (77.99, 127.37),
 (49.34, 76.53),
 (22.45, 31.88),
 (60.56, 96.13),
 (108.07, 183.1),
 (42.47, 64.78),
 (

**Save Model and Params**

In [63]:
# Save best model and cv
save_model(best_model_svm_reg, title="best_model_svm_reg_02", save="joblib")
save_model(grid_svm_reg, title="best_cv_svm_reg_02", save="joblib")

## Reg Model 3: Random Forest

In [984]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_rf_reg = Pipeline([('preprocessor', preprocessor),
                            ('rf_reg',
                             RandomForestRegressor(n_estimators=1500,
                                                   max_features='sqrt',
                                                   random_state=random_state,
                                                   max_depth=4,
                                                   min_samples_split=10,
                                                   min_samples_leaf=1,
                                                   bootstrap=False,
                                                   n_jobs=-1))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [962]:
# Display possible hyperparameters for XGBoost Regressor
test_rf_reg = RandomForestRegressor()
test_rf_reg.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

**Default values for Random Forest Regressor** (as base for hyperparameter search):

n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None

In [963]:
# Define hyperparameter distribution
param_distribs_rf_reg = {
    'rf_reg__n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'rf_reg__max_features': ['auto', 'sqrt'],
    'rf_reg__max_depth': [None, 1, 2, 3, 4, 5, 7, 10, 15, 20, 30, 40, 50, 75, 100],
    'rf_reg__min_samples_split': [2, 5, 10],
    'rf_reg__min_samples_leaf': [1, 2, 4],
    'rf_reg__bootstrap': [True, False]
}

In [964]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_rf_reg = RandomizedSearchCV(pipeline_rf_reg,
                                 param_distribs_rf_reg,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=10,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_rf_reg = rnd_rf_reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 35.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 45.8min finished


In [965]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_rf_reg.best_score_))
print("Best parameters:\n{}".format(rnd_rf_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_rf_reg.best_estimator_))

Best score:
-0.23
Best parameters:
{'rf_reg__n_estimators': 1000, 'rf_reg__min_samples_split': 10, 'rf_reg__min_samples_leaf': 1, 'rf_reg__max_features': 'sqrt', 'rf_reg__max_depth': 100, 'rf_reg__bootstrap': False}


### Hyperparameter Tuning with GridSearchCV

In [1006]:
# Define hyperparameter grid
param_grid_rf_reg = {
#    'rf_reg__n_estimators': [1200, 2000],
#    'rf_reg__max_features': ['auto', 'sqrt'],
    'rf_reg__max_depth': [10, 15],
    'rf_reg__min_samples_split': [6, 10],
#    'rf_reg__min_samples_leaf': [1, 2],
#    'rf_reg__bootstrap': [True, False]
}

In [1007]:
# Create and fit GridSearchCV, save "best_model"
grid_rf_reg = GridSearchCV(pipeline_rf_reg,
                            param_grid_rf_reg,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_rf_reg = grid_rf_reg.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.0min finished


In [1008]:
# Save best model
best_model_rf_reg = grid_rf_reg.best_estimator_['rf_reg']

In [1009]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_rf_reg.best_score_))
print("Best parameters:\n{}".format(grid_rf_reg.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
-0.24
Best parameters:
{'rf_reg__max_depth': 15, 'rf_reg__min_samples_split': 6}


### Feature Importances

In [1010]:
# Display feature importances
fi_rf_reg = get_feat_importances(best_model_rf_reg)
fi_rf_reg

,weight
room_type_Private room,0.227147
accommodates,0.138154
bedrooms,0.113998
calc_host_lst_count_sqrt_log,0.045111
am_tv,0.042288
bathrooms_log,0.037592
availability_90,0.033099
accommodates_per_bed,0.032034
am_elevator,0.027565
minimum_nights_log,0.023683


### Final Evaluation Best Model

In [892]:
# Load existing model
#load_best_model = load_model(title="best_model_rf_reg_01", dataset_loc=dataset_loc, dataset_date=dataset_date)
#load_best_cv = load_model(title="best_cv_rf_reg_01", dataset_loc=dataset_loc, dataset_date=dataset_date)

[20:07:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


**Learning Curves (Overfitting)**

**Training Set**

In [1011]:
# Predict target with "best model"
y_train_pred_rf_reg = best_model_rf_reg.predict(X_train_prep)

In [1012]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_rf_reg, model="reg")

MSE: 0.10
RMSE: 0.31
MAE: 0.25
R2: 0.70
MAPE: 6.31
MAPE median: 5.07


**Testing Set**

In [1013]:
# Predict target with "best model"
y_test_pred_rf_reg = best_model_rf_reg.predict(X_test_prep)

In [1014]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_rf_reg, model="reg")

MSE: 0.15
RMSE: 0.38
MAE: 0.29
R2: 0.54
MAPE: 7.61
MAPE median: 5.83


In [1003]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_rf_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.36322871, 0.39248337])

**Median Price Intervals**

In [1004]:
# Save MAPE_median as variable
MAPE_median_rf_reg = (median_absolute_percentage_error(y_test, y_test_pred_rf_reg))/100

In [1005]:
# Calculate price interval for MAPE median
y_pred_interval_rf_reg = tuple([(round(math.exp(el-el*MAPE_median_rf_reg),2),round(math.exp(el+el*MAPE_median_rf_reg),2)) for el in y_test_pred_rf_reg])
y_pred_interval_rf_reg

((52.49, 84.34),
 (63.69, 104.73),
 (48.79, 77.71),
 (31.74, 48.02),
 (37.33, 57.58),
 (31.18, 47.07),
 (40.96, 63.88),
 (39.19, 60.8),
 (50.29, 80.39),
 (43.01, 67.48),
 (25.4, 37.42),
 (40.63, 63.31),
 (60.1, 98.14),
 (26.03, 38.45),
 (34.87, 53.35),
 (65.81, 108.65),
 (68.9, 114.37),
 (52.99, 85.24),
 (57.08, 92.64),
 (47.3, 75.06),
 (85.92, 146.45),
 (50.15, 80.13),
 (42.29, 66.21),
 (76.92, 129.38),
 (25.16, 37.02),
 (50.02, 79.91),
 (76.51, 128.6),
 (33.78, 51.48),
 (28.21, 42.09),
 (45.88, 72.54),
 (26.96, 40.0),
 (56.11, 90.88),
 (55.91, 90.52),
 (29.42, 44.11),
 (44.83, 70.68),
 (58.97, 96.07),
 (36.57, 56.26),
 (51.65, 82.83),
 (33.59, 51.16),
 (32.83, 49.87),
 (31.65, 47.86),
 (25.48, 37.55),
 (38.19, 59.07),
 (35.32, 54.12),
 (30.45, 45.84),
 (46.57, 73.76),
 (47.55, 75.5),
 (72.92, 121.87),
 (59.19, 96.49),
 (90.86, 155.9),
 (34.06, 51.96),
 (66.47, 109.87),
 (52.47, 84.3),
 (48.38, 76.98),
 (35.34, 54.15),
 (46.65, 73.9),
 (57.55, 93.5),
 (33.75, 51.44),
 (53.35, 85.89),


**Save Model and Params**

In [1041]:
# Save best model and cv
save_model(best_model_rf_reg, title="best_model_rf_reg_02", save="joblib")
save_model(grid_rf_reg, title="best_cv_rf_reg_02", save="joblib")

## Reg Model 4: CatBoost

In [46]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_cat_reg = Pipeline([('preprocessor', preprocessor),
                             ('cat_reg',
                              CatBoostRegressor(n_estimators=150,
                                                learning_rate=0.3,
                                                l2_leaf_reg=4,
                                                loss_function="RMSE",
                                                random_state=random_state,
                                                depth=4))])

### Hyperparameter Pre-Tuning with RandomizedSearchCV

In [79]:
# Display possible hyperparameters for XGBoost Regressor
test_cat_reg = CatBoostRegressor()
test_cat_reg.get_params().keys()

dict_keys(['loss_function'])

**Default values for CatBoostRegressor** (as base for hyperparameter search):

iterations=None, learning_rate=None, depth=None, l2_leaf_reg=None, model_size_reg=None, rsm=None, loss_function='RMSE', border_count=None, feature_border_type=None, per_float_feature_quantization=None, input_borders=None, output_borders=None, fold_permutation_block=None, od_pval=None, od_wait=None, od_type=None, nan_mode=None, counter_calc_method=None, leaf_estimation_iterations=None, leaf_estimation_method=None, thread_count=None, random_seed=None, use_best_model=None, best_model_min_trees=None, verbose=None, silent=None, logging_level=None, metric_period=None, ctr_leaf_count_limit=None, store_all_simple_ctr=None, max_ctr_complexity=None, has_time=None, allow_const_label=None, target_border=None, one_hot_max_size=None, random_strength=None, name=None, ignored_features=None, train_dir=None, custom_metric=None, eval_metric=None, bagging_temperature=None, save_snapshot=None, snapshot_file=None, snapshot_interval=None, fold_len_multiplier=None, used_ram_limit=None, gpu_ram_part=None, pinned_memory_size=None, allow_writing_files=None, final_ctr_computation_mode=None, approx_on_full_history=None, boosting_type=None, simple_ctr=None, combinations_ctr=None, per_feature_ctr=None, ctr_description=None, ctr_target_border_count=None, task_type=None, device_config=None, devices=None, bootstrap_type=None, subsample=None, mvs_reg=None, sampling_frequency=None, sampling_unit=None, dev_score_calc_obj_block_size=None, dev_efb_max_buckets=None, sparse_features_conflict_fraction=None, max_depth=None, n_estimators=None, num_boost_round=None, num_trees=None, colsample_bylevel=None, random_state=None, reg_lambda=None, objective=None, eta=None, max_bin=None, gpu_cat_features_storage=None, data_partition=None, metadata=None, early_stopping_rounds=None, cat_features=None, grow_policy=None, min_data_in_leaf=None, min_child_samples=None, max_leaves=None, num_leaves=None, score_function=None, leaf_estimation_backtracking=None, ctr_history_unit=None, monotone_constraints=None, feature_weights=None, penalties_coefficient=None, first_feature_use_penalties=None, per_object_feature_penalties=None, model_shrink_rate=None, model_shrink_mode=None, langevin=None, diffusion_temperature=None, boost_from_average=None

In [43]:
# Define hyperparameter distribution
param_distribs_cat_reg = {
    'cat_reg__n_estimators': randint(low=130, high=180),    # initial: randint(low=10, high=200)
    'cat_reg__l2_leaf_reg': randint(low=2, high=11),       # initial: randint(low=1, high=15)
    'cat_reg__depth': randint(low=4, high=6),             # initial: randint(low=1, high=15)
    'cat_reg__learning_rate': [0.15, 0.18, 0.2, 0.22, 0.25, 0.27, 0.3] # initial: [0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
}

In [44]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_cat_reg = RandomizedSearchCV(pipeline_cat_reg,
                                 param_distribs_cat_reg,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=15,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_cat_reg = rnd_cat_reg.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   32.2s finished


0:	learn: 0.5156745	total: 56.2ms	remaining: 8.38s
1:	learn: 0.4715264	total: 59.1ms	remaining: 4.38s
2:	learn: 0.4446813	total: 61.6ms	remaining: 3.02s
3:	learn: 0.4253420	total: 63.3ms	remaining: 2.31s
4:	learn: 0.4137535	total: 65ms	remaining: 1.88s
5:	learn: 0.4061849	total: 66.6ms	remaining: 1.6s
6:	learn: 0.3988524	total: 68.2ms	remaining: 1.39s
7:	learn: 0.3923261	total: 70.5ms	remaining: 1.25s
8:	learn: 0.3884186	total: 72.6ms	remaining: 1.14s
9:	learn: 0.3844157	total: 74.9ms	remaining: 1.05s
10:	learn: 0.3792050	total: 76.5ms	remaining: 967ms
11:	learn: 0.3759227	total: 78.2ms	remaining: 899ms
12:	learn: 0.3738849	total: 79.8ms	remaining: 841ms
13:	learn: 0.3711826	total: 84.6ms	remaining: 822ms
14:	learn: 0.3690398	total: 86.8ms	remaining: 782ms
15:	learn: 0.3676172	total: 90.2ms	remaining: 756ms
16:	learn: 0.3664609	total: 93.6ms	remaining: 732ms
17:	learn: 0.3652486	total: 96.9ms	remaining: 711ms
18:	learn: 0.3636378	total: 102ms	remaining: 701ms
19:	learn: 0.3626866	total

In [45]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_cat_reg.best_score_))
print("Best parameters:\n{}".format(rnd_cat_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_cat_reg.best_estimator_))

Best score:
-0.21
Best parameters:
{'cat_reg__depth': 4, 'cat_reg__l2_leaf_reg': 4, 'cat_reg__learning_rate': 0.3, 'cat_reg__n_estimators': 150}


### Hyperparameter Tuning with GridSearchCV

In [48]:
# Define hyperparameter grid
param_grid_cat_reg = {
#    'cat_reg__n_estimators': [150, 155],
#    'cat_reg__l2_leaf_reg': [3, 4],
    'cat_reg__depth': [4, 5],
#    'cat_reg__learning_rate': [0.15, 0.2, 0.25]
}

In [49]:
# Create and fit GridSearchCV, save "best_model"
grid_cat_reg = GridSearchCV(pipeline_cat_reg,
                            param_grid_cat_reg,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_cat_reg = grid_cat_reg.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    6.7s remaining:    4.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    7.5s finished


0:	learn: 0.5156745	total: 2.05ms	remaining: 306ms
1:	learn: 0.4715264	total: 3.86ms	remaining: 286ms
2:	learn: 0.4446813	total: 5.6ms	remaining: 274ms
3:	learn: 0.4253420	total: 7.27ms	remaining: 266ms
4:	learn: 0.4137535	total: 8.93ms	remaining: 259ms
5:	learn: 0.4061849	total: 10.5ms	remaining: 253ms
6:	learn: 0.3988524	total: 12.2ms	remaining: 249ms
7:	learn: 0.3923261	total: 13.8ms	remaining: 245ms
8:	learn: 0.3884186	total: 15.5ms	remaining: 243ms
9:	learn: 0.3844157	total: 17.2ms	remaining: 241ms
10:	learn: 0.3792050	total: 18.8ms	remaining: 238ms
11:	learn: 0.3759227	total: 20.5ms	remaining: 236ms
12:	learn: 0.3738849	total: 22.1ms	remaining: 233ms
13:	learn: 0.3711826	total: 23.8ms	remaining: 231ms
14:	learn: 0.3690398	total: 25.4ms	remaining: 229ms
15:	learn: 0.3676172	total: 27ms	remaining: 226ms
16:	learn: 0.3664609	total: 28.9ms	remaining: 226ms
17:	learn: 0.3652486	total: 31.7ms	remaining: 232ms
18:	learn: 0.3636378	total: 34.4ms	remaining: 237ms
19:	learn: 0.3626866	tota

In [50]:
# Save best model
best_model_cat_reg = grid_cat_reg.best_estimator_['cat_reg']

In [51]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_cat_reg.best_score_))
print("Best parameters:\n{}".format(grid_cat_reg.best_params_))
#print("Best estimator:\n{}".format(grid_xgb_reg.best_estimator_))

Best score:
-0.21
Best parameters:
{'cat_reg__depth': 4}


### Feature Importances

In [52]:
# Display feature importances
fi_cat_reg = get_feat_importances(best_model_cat_reg)
fi_cat_reg

,weight
room_type_Private room,26.286565
guests_included_calc,16.531309
bedrooms,8.930682
calc_host_lst_count_sqrt_log,7.459091
minimum_nights_log,4.374038
am_elevator,3.976968
property_type_Boutique hotel,3.644855
room_type_Shared room,3.116251
bathrooms_log,2.742996
accommodates_per_bed,2.285218


### Final Evaluation Best Model

In [53]:
# Load existing model
#load_best_model = load_model(title="best_model_xgb_reg", dataset_loc=dataset_loc, dataset_date=dataset_date)
#load_best_cv = load_model(title="best_cv_xgb_reg", dataset_loc=dataset_loc, dataset_date=dataset_date)

**Learning Curves (Overfitting)**

**Training Set**

In [54]:
# Predict target with "best model"
y_train_pred_cat_reg = best_model_cat_reg.predict(X_train_prep)

In [55]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_cat_reg, model="reg")

MSE: 0.10
RMSE: 0.32
MAE: 0.25
R2: 0.70
MAPE: 6.24
MAPE median: 4.87


**Testing Set**

In [56]:
# Predict target with "best model"
y_test_pred_cat_reg = best_model_cat_reg.predict(X_test_prep)

In [57]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_cat_reg, model="reg")

MSE: 0.13
RMSE: 0.35
MAE: 0.27
R2: 0.64
MAPE: 6.69
MAPE median: 5.07


In [58]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_cat_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.34024543, 0.36782985])

**Median Price Intervals**

In [59]:
# Save MAPE_median as variable
MAPE_median_cat_reg = (median_absolute_percentage_error(y_test, y_test_pred_cat_reg))/100

In [60]:
# Calculate price interval for MAPE median
y_pred_interval_cat_reg = tuple([(round(math.exp(el-el*MAPE_median_cat_reg),2),round(math.exp(el+el*MAPE_median_cat_reg),2)) for el in y_test_pred_cat_reg])
y_pred_interval_cat_reg

((74.42, 117.88),
 (54.15, 82.91),
 (42.66, 63.68),
 (26.86, 38.16),
 (53.28, 81.45),
 (39.16, 57.93),
 (38.41, 56.69),
 (75.17, 119.21),
 (93.75, 152.21),
 (39.94, 59.2),
 (64.4, 100.44),
 (38.96, 57.59),
 (34.15, 49.79),
 (128.14, 215.1),
 (35.39, 51.79),
 (72.16, 113.93),
 (35.11, 51.32),
 (130.84, 220.13),
 (26.92, 38.26),
 (62.3, 96.83),
 (68.14, 106.92),
 (67.52, 105.86),
 (23.91, 33.55),
 (76.4, 121.37),
 (42.98, 64.22),
 (38.72, 57.2),
 (78.43, 124.93),
 (68.56, 107.65),
 (74.62, 118.23),
 (48.12, 72.76),
 (103.45, 169.73),
 (34.28, 49.98),
 (35.23, 51.53),
 (28.96, 41.47),
 (52.23, 79.66),
 (120.61, 201.15),
 (37.03, 54.45),
 (36.65, 53.83),
 (27.67, 39.44),
 (37.01, 54.42),
 (48.75, 73.81),
 (41.6, 61.93),
 (53.93, 82.54),
 (24.34, 34.22),
 (80.08, 127.85),
 (55.95, 85.97),
 (116.33, 193.27),
 (103.88, 170.52),
 (26.92, 38.25),
 (35.16, 51.41),
 (51.75, 78.85),
 (51.3, 78.1),
 (83.59, 134.06),
 (57.98, 89.42),
 (23.16, 32.38),
 (61.07, 94.73),
 (124.19, 207.77),
 (37.23, 54.7

**Save Model and Params**

In [61]:
# Save best model and cv
save_model(best_model_cat_reg, title="best_model_cat_reg_02", save="joblib")
save_model(grid_cat_reg, title="best_cv_cat_reg_02", save="joblib")

## NN Model 1: Neural Networks

In [62]:
# Build the model
model_nn_seq = models.Sequential()

# Input Layer
model_nn_seq.add(
    layers.Dense(128,
                 input_shape=(X_train_prep.shape[1], ),
                 kernel_regularizer=regularizers.l1(0.005),
                 activation='relu'))

# Hidden Layers
model_nn_seq.add(
    layers.Dense(256,
                 kernel_regularizer=regularizers.l1(0.005),
                 activation='relu'))
model_nn_seq.add(
    layers.Dense(256,
                 kernel_regularizer=regularizers.l1(0.005),
                 activation='relu'))
model_nn_seq.add(
    layers.Dense(512,
                 kernel_regularizer=regularizers.l1(0.005),
                 activation='relu'))

# Output Layer
model_nn_seq.add(layers.Dense(1, activation='linear'))

# Compile the model
model_nn_seq.compile(loss='mean_absolute_percentage_error',
                     optimizer='adam',
                     metrics=['mean_absolute_percentage_error'])

# Model summary
print(model_nn_seq.summary())

# Visualize the neural network
#SVG(model_to_dot(model_nn_seq, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               15104     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 513       
Total params: 246,017
Trainable params: 246,017
Non-trainable params: 0
_________________________________________________________________
None


In [1332]:
# Create pipeline with preprocessing
#pipeline_nn_reg = Pipeline([('preprocessor', preprocessor),
#                             ('nn_reg',
#                              KerasRegressor(build_fn=model_nn_seq, epochs=20, batch_size=250))])

In [1333]:
# Define hyperparameter grid
#param_grid_nn_reg = {
#    'xgb_reg__bootstrap': [True, False],
#    'nn_reg__epochs': [10, 20]
#}

In [1334]:
# Create and fit GridSearchCV, save "best_model"
#grid_nn_reg = GridSearchCV(pipeline_nn_reg,
#                            param_grid_nn_reg,
#                            cv=5,
#                            scoring=scoring,
#                            return_train_score=True,
#                            verbose=4,
#                            n_jobs=-1)

#grid_nn_reg = grid_nn_reg.fit(X_train, y_train)

In [63]:
# Train the model
#model_nn_seq_start = time.time()

best_model_nn_reg = model_nn_seq.fit(X_train_prep,
                                        y_train,
                                        epochs=20,
                                        batch_size=256,
                                        validation_split=0.2)

#model_nn_seq_end = time.time()

#print(f"Time taken to run: {round((model_nn_seq_end - model_nn_seq_start)/60,1)} minutes")

Train on 6687 samples, validate on 1672 samples
Epoch 1/20
6687/6687 [==============================] - 2s 314us/step - loss: 89.7828 - mean_absolute_percentage_error: 31.1804 - val_loss: 66.8553 - val_mean_absolute_percentage_error: 13.3454
Epoch 2/20
6687/6687 [==============================] - 1s 111us/step - loss: 60.5978 - mean_absolute_percentage_error: 11.3438 - val_loss: 55.4232 - val_mean_absolute_percentage_error: 10.7918
Epoch 3/20
6687/6687 [==============================] - 1s 102us/step - loss: 50.6808 - mean_absolute_percentage_error: 9.4313 - val_loss: 47.3532 - val_mean_absolute_percentage_error: 9.5435
Epoch 4/20
6687/6687 [==============================] - 1s 109us/step - loss: 43.7905 - mean_absolute_percentage_error: 8.2567 - val_loss: 42.2621 - val_mean_absolute_percentage_error: 9.0747
Epoch 5/20
6687/6687 [==============================] - 1s 110us/step - loss: 39.3247 - mean_absolute_percentage_error: 7.8196 - val_loss: 38.0525 - val_mean_absolute_percentage_er

**Training Set**

In [64]:
# Predict target with "best model"
y_train_pred_nn_reg = model_nn_seq.predict(X_train_prep).ravel()

In [65]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_nn_reg, model="reg")

MSE: 0.12
RMSE: 0.35
MAE: 0.24
R2: 0.65
MAPE: 6.00
MAPE median: 4.32


**Testing Set**

In [66]:
# Predict target with "best model"
y_test_pred_nn_reg = model_nn_seq.predict(X_test_prep).ravel()

In [67]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_nn_reg, model="reg")

MSE: 0.19
RMSE: 0.43
MAE: 0.30
R2: 0.46
MAPE: 7.28
MAPE median: 5.71


In [68]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_nn_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.33338371, 0.5163427 ])

**Median Price Intervals**

In [69]:
# Save MAPE_median as variable
MAPE_median_nn_reg = (median_absolute_percentage_error(y_test, y_test_pred_nn_reg))/100

In [70]:
# Calculate price interval for MAPE median
y_pred_interval_nn_reg = tuple([(round(math.exp(el-el*MAPE_median_nn_reg),2),round(math.exp(el+el*MAPE_median_nn_reg),2)) for el in y_test_pred_nn_reg])
y_pred_interval_nn_reg

((76.21, 128.78),
 (47.52, 75.83),
 (42.57, 67.03),
 (35.37, 54.47),
 (52.86, 85.45),
 (31.0, 46.99),
 (49.25, 78.93),
 (58.77, 96.22),
 (88.07, 151.45),
 (35.64, 54.93),
 (49.49, 79.36),
 (42.4, 66.73),
 (25.6, 37.91),
 (119.62, 213.46),
 (39.5, 61.65),
 (49.85, 80.01),
 (40.22, 62.91),
 (121.34, 216.91),
 (23.29, 34.09),
 (54.51, 88.45),
 (64.33, 106.49),
 (59.03, 96.71),
 (19.32, 27.65),
 (60.08, 98.63),
 (33.06, 50.49),
 (37.39, 57.97),
 (82.24, 140.25),
 (66.38, 110.3),
 (61.23, 100.76),
 (42.61, 67.12),
 (89.37, 153.95),
 (32.02, 48.72),
 (33.87, 51.88),
 (33.27, 50.85),
 (58.86, 96.39),
 (82.13, 140.04),
 (27.08, 40.36),
 (37.42, 58.02),
 (22.35, 32.56),
 (29.78, 44.91),
 (36.74, 56.84),
 (43.94, 69.46),
 (51.04, 82.15),
 (24.64, 36.32),
 (63.25, 104.49),
 (47.15, 75.17),
 (100.33, 175.28),
 (117.78, 209.79),
 (24.19, 35.57),
 (33.99, 52.08),
 (45.11, 71.53),
 (47.77, 76.29),
 (80.43, 136.8),
 (50.81, 81.74),
 (25.09, 37.07),
 (60.44, 99.3),
 (134.47, 243.4),
 (36.31, 56.09),
 (

In [71]:
# Evaluate the model
#model_nn_seq.model_evaluation(model_nn_seq, skip_epochs=2, X_train=X_train, X_test=X_test)

#score_nn_seq = model_nn_seq.evaluate(X_train_prep, y_train,verbose=1)
#print(score_nn_seq)

**Save Model and Params**

In [72]:
# Save best model and cv
save_model(best_model_nn_reg, title="best_model_nn_reg_02", save="joblib")

## Final Evaluation with Testing Set

In [ ]:
best_models_reg = [best_model_xgb_reg, best_model_svm_reg]

In [ ]:
# Transform X_test for final evaluation
#X_test_prep = preprocessor.transform(X_test)

In [ ]:
# Predict target with "best model"
#y_pred_rf_reg = best_model_rf_reg.predict(X_test_prep)

In [ ]:
# Final evaluation of "best model"
#print("MSE: {:.2f}".format(mean_squared_error(y_test, y_pred_rf_reg))),
#print("RMSE: {:.2f}".format(mean_squared_error(y_test, y_pred_rf_reg, squared=False))),
#print("MAE: {:.2f}".format(mean_absolute_error(y_test, y_pred_rf_reg))),
#print("R2: {:.2f}".format(r2_score(y_test, y_pred_rf_reg))),

In [ ]:
# Illustrate best model
#fig, axes = plt.subplots(1, 2, figsize = (14, 6))
#axes = axes.flatten()

#y_pred = best_model.predict(X_test_prep)
#axes[0].scatter(y_test, y_pred)
#axes[0].set_xlabel('y_test')
#axes[0].set_ylabel('y_pred')

#coef = best_model.best_estimator_.named_steps['xgb'].coef_
#mean_coef = np.mean(coef)
#axes[1].plot(coef, 'o')
#axes[1].set_xlabel('coefficient index')
#axes[1].set_ylabel('coefficient size')
#axes[1].axhline(y = mean_coef, color = 'red', linestyle = '--', alpha = 0.5)
#plt.show()

In [ ]:
# Display confidence interval (scipy stats)
#confidence = 0.95
#squared_errors = (y_pred_rf_reg - y_test) ** 2
#np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
#                         loc=squared_errors.mean(),
#                         scale=stats.sem(squared_errors)))

# Future Work

**Predictive modeling**
- Apply further models and adapt current ones (e.g. NN)
- Examine other prediction targets (e.g. occupancy rate)

**Feature engineering**
- Explore NLP for text fields (descriptions, reviews, ...)
- Scrape listing photos and analyze quality
- Enhance current feature set

**Lean structure**
- Remove remaining redundancies wherever possible (e.g. pack repeated steps into functions, apply more pipelines, ...)

**Cloud**
- Move both model creation and app into the cloud (GCP)

**Automatization and replicability**
- Build a workflow to automatically retrain model monthly with new datasets
- Use automated outlier detection
- Use automated feature engineering

**Replicability**
- Reduce redundancies wherever possible (e.g. pack repeated steps into functions, apply more pipelines, ...)
- Apply analysis to other cities and compare results